In [ ]:
import feedparser
import requests
from datetime import datetime

# Fonction pour convertir une date Feedparser en ISO 8601
def convert_to_iso8601(date_str):
    try:
        # Parse la date avec le format de Feedparser
        parsed_date = datetime.strptime(date_str, "%a, %d %b %Y %H:%M:%S %z")
        # Convertit en format ISO 8601
        return parsed_date.isoformat()
    except ValueError:
        return None  # Retourne None si la date est invalide ou absente



#Liste des mots-clés pour filtrer les articles
keywords = ['réseau', 'sécurité informatique', 'numérique']

# Liste des flux RSS (Feedly et Google Alerts)
rss_feeds = [
    "http://feeds.feedburner.com/Grafikart",  # liens feedly 
    "http://blog.openclassrooms.com/feed/",
    "https://www.google.com/alerts/feeds/00304130937566327789/2520171459693073861",     # Google Alert
]

# Notion API Configuration
NOTION_API_URL = "https://api.notion.com/v1/pages"
NOTION_API_KEY = "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"  # Remplace par ton Token Notion
DATABASE_ID = "144359de043f80359769f4296f5cef97"  # Remplace par l'ID de ta base de données

# Fonction pour analyser les flux RSS
def parse_rss_feeds(feeds):
    articles = []
    for feed_url in feeds:
        print(f"Analyse du flux RSS : {feed_url}")
        feed = feedparser.parse(feed_url)
        
        # Vérifie si le flux est valide
        if 'title' in feed.feed:
            print(f"Flux trouvé : {feed.feed.title}")
        else:
            print("Erreur : Flux non valide.")
            continue
        
        for entry in feed.entries:
            article = {
                "title": entry.title,                
                "link": entry.link,
                "published": entry.published if "published" in entry else "N/A",
                "summary": entry.summary if "summary" in entry else "N/A"
            }
            articles.append(article)
    
    return articles

# Fonction pour envoyer un article à Notion
def add_article_to_notion(article):
    headers = {
        "Authorization": f"Bearer {NOTION_API_KEY}",
        "Content-Type": "application/json",
        "Notion-Version": "2022-06-28"
    }
    # Conversion de la date
    iso_date = convert_to_iso8601(article["published"])

    data = {
        "parent": {"database_id": DATABASE_ID},
        "properties": {
            "Title": {"title": [{"text": {"content": article["title"]}}]},            
            "Link": {"url": article["link"]},
            "Published": {"date": {"start": iso_date} if iso_date else None},
            "Summary": {"rich_text": [{"text": {"content": article["summary"]}}]}       
        }
    }
    response = requests.post(NOTION_API_URL, headers=headers, json=data)
    if response.status_code == 200:
        print(f"Article ajouté à Notion : {article['title']}")
    else:
        print(f"Erreur lors de l'ajout à Notion : {response.status_code}, {response.text}")


# Fonction principale
if __name__ == "__main__":
    # Étape 1 : Analyse des flux RSS
    collected_articles = parse_rss_feeds(rss_feeds)
    
    # Étape 3 : Envoi à Notion
    print(f"\nAjout des articles à Notion...")
    for article in collected_articles:
        add_article_to_notion(article)

    print(f"\n{len(collected_articles)} articles ajoutés à Notion.")

Analyse du flux RSS : http://feeds.feedburner.com/Grafikart
Flux trouvé : Grafikart.fr
Analyse du flux RSS : http://blog.openclassrooms.com/feed/
Flux trouvé : OpenClassrooms : le blog
Analyse du flux RSS : https://www.google.com/alerts/feeds/00304130937566327789/2520171459693073861
Flux trouvé : Google Alert - Cyber sécurité

Ajout des articles à Notion...
Article ajouté à Notion : Les évolutions de l'HTTP
Article ajouté à Notion : Interview avec Benjamin Code
Article ajouté à Notion : Interview avec ledevultime
Article ajouté à Notion : TP : Grille photos adaptative
Article ajouté à Notion : Svelte 5 & les runes
Article ajouté à Notion : L’alternance, un essai couronné de succès
Article ajouté à Notion : La culture DevOps : au croisement de la tech et de l’humain
Article ajouté à Notion : Découvrez l’univers de la cybersécurité : entretien avec Yann Bonnet
Article ajouté à Notion : Data Science : ces experts de la prise  de décision stratégique en entreprise
Article ajouté à Notion :

In [ ]:
import feedparser
import requests
from datetime import datetime

# Fonction pour convertir une date Feedparser en ISO 8601
def convert_to_iso8601(date_str):
    try:
        parsed_date = datetime.strptime(date_str, "%a, %d %b %Y %H:%M:%S %z")
        return parsed_date.isoformat()
    except ValueError:
        return None

# Liste des mots-clés pour filtrer les articles
keywords = ['réseau', 'sécurité informatique', 'numérique']

# Liste des flux RSS (Feedly et Google Alerts)
rss_feeds = [
    "http://feeds.feedburner.com/Grafikart",
    "http://blog.openclassrooms.com/feed/",
    "https://www.google.com/alerts/feeds/00304130937566327789/2520171459693073861",
]

# Notion API Configuration
NOTION_API_URL = "https://api.notion.com/v1/pages"
NOTION_API_KEY = "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"  # Token de Notion
DATABASE_ID = "144359de043f80359769f4296f5cef97"  # ID de ta base de données

# Fonction pour analyser les flux RSS avec filtrage
def parse_rss_feeds(feeds):
    articles = []
    for feed_url in feeds:
        print(f"Analyse du flux RSS : {feed_url}")
        feed = feedparser.parse(feed_url)
        
        if 'title' in feed.feed:
            print(f"Flux trouvé : {feed.feed.title}")
        else:
            print("Erreur : Flux non valide.")
            continue
        
        for entry in feed.entries:
            # Vérification des mots-clés dans le titre et le résumé
            if any(keyword in entry.title.lower() or keyword in entry.summary.lower() for keyword in keywords):
                article = {
                    "title": entry.title,
                    "link": entry.link,
                    "published": entry.published if "published" in entry else "N/A",
                    "summary": entry.summary if "summary" in entry else "N/A"
                }
                articles.append(article)

    return articles

# Fonction pour envoyer un article à Notion
def add_article_to_notion(article):
    headers = {
        "Authorization": f"Bearer {NOTION_API_KEY}",
        "Content-Type": "application/json",
        "Notion-Version": "2022-06-28"
    }
    iso_date = convert_to_iso8601(article["published"])

    data = {
        "parent": {"database_id": DATABASE_ID},
        "properties": {
            "Title": {"title": [{"text": {"content": article["title"]}}]},
            "Link": {"url": article["link"]},
            "Published": {"date": {"start": iso_date} if iso_date else None},
            "Summary": {"rich_text": [{"text": {"content": article["summary"]}}]}
        }
    }
    response = requests.post(NOTION_API_URL, headers=headers, json=data)
    if response.status_code == 200:
        print(f"Article ajouté à Notion : {article['title']}")
    else:
        print(f"Erreur lors de l'ajout à Notion : {response.status_code}, {response.text}")

# Fonction principale
if __name__ == "__main__":
    # Étape 1 : Analyse des flux RSS
    collected_articles = parse_rss_feeds(rss_feeds)
    
    # Étape 3 : Envoi à Notion
    print(f"\nAjout des articles à Notion...")
    for article in collected_articles:
        add_article_to_notion(article)

    print(f"\n{len(collected_articles)} articles ajoutés à Notion.")

Analyse du flux RSS : http://feeds.feedburner.com/Grafikart
Flux trouvé : Grafikart.fr
Analyse du flux RSS : http://blog.openclassrooms.com/feed/
Flux trouvé : OpenClassrooms : le blog
Analyse du flux RSS : https://www.google.com/alerts/feeds/00304130937566327789/2520171459693073861
Flux trouvé : Google Alert - Cyber sécurité

Ajout des articles à Notion...
Article ajouté à Notion : La culture DevOps : au croisement de la tech et de l’humain
Article ajouté à Notion : UX designer, un métier de plus en plus convoité
Article ajouté à Notion : Numérique : nouvelle éducation, nouveau travail, nouvelle vie ?
Article ajouté à Notion : Renforcer la <b>cybersécurité</b> en Afrique: Symposium à Ifrane sur l'identité numérique
Article ajouté à Notion : IA, <b>cybersécurité</b>, mobilité… Les défis numériques du maire de Nevers et des villes médianes
Article ajouté à Notion : <b>Cybersécurité</b> des secteurs critiques: le défi Nis2 | Paperjam News
Article ajouté à Notion : MonAideCyber : la métho